In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib as mpl

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

## Setup

In [3]:
n = 1000
Λ = np.linspace(1,1e1,n)
b = np.ones(n,dtype=np.longdouble)
b /= np.linalg.norm(b)

λmin = np.min(Λ).astype(np.double)
λmax = np.max(Λ).astype(np.double)
kappa = λmax / λmin

λmin_ = λmin/2
λmax_ = 2*λmax

In [4]:
w = 0

# norm to measure function approximation and linear system errors
B = Λ

A = np.diag(Λ)

In [5]:
K = 41
Q,(a_,b_) = lanczos_reorth(A,b,K,reorth=K)

In [6]:
plot_tol = 1e-14

In [7]:
T = np.zeros((3,K))
T[0,1:] = b_[:-1]
T[1] = a_-w
T[2,:-1] = b_[:-1]

e1 = np.zeros(K+1)
e1[0] = 1

In [8]:
err_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = (1/(Λ-w))*b - np.linalg.norm(b)*Q[:,:k+1]@sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1])
        err_CG[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_CG[k] < plot_tol:
        break

In [9]:
res_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        r = b - np.linalg.norm(b)*(Λ-w)*(Q[:,:k+1]@sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1]))
        res_CG[k] = np.sqrt( r@r )
    except: pass
    
    if res_CG[k] < plot_tol:
        break

In [10]:
Anorm_estimate = np.full(K,np.nan)
Anorm_bound = np.full(K,np.nan)

d = 10

for k in range(0,K):
    try:
        Tk_inv_e1 = sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1])
        e = (1/(Λ-w))*b - np.linalg.norm(b)*Q[:,:k+1]@Tk_inv_e1
                
        if k+d+1<K:
            Tkd_inv_e1 = sp.linalg.solve_banded((1,1),T[:,:k+d+1],e1[:k+d+1])
            
        Anorm_estimate[k] = np.linalg.norm(b)*np.sqrt(e1[:k+d+1]@Tkd_inv_e1 - e1[:k+1]@Tk_inv_e1)
        Anorm_bound[k] = np.sqrt(np.linalg.norm(b)**2*(e1[:k+d+1]@Tkd_inv_e1 - e1[:k+1]@Tk_inv_e1) + (1/λmin) * res_CG[k+d+1]**2)

        
    except: pass
    
    if err_CG[k] < plot_tol:
        break

## $f(x) = \sqrt{x}$, $\Gamma =$ circle

In [11]:
def f(x):
    return np.sqrt(x)

fAb = f(Λ)*b

In [12]:
err_lanczos = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = fAb - lanczos_FA(f,Q.astype(np.double),a_,b_,k,normb=np.linalg.norm(b))
        err_lanczos[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_lanczos[k] < plot_tol:
        break

In [13]:
a_priori_bound_circle = np.full(K,np.inf)
a_posteriori_bound_circle = np.full(K,np.inf)
exact_bound_circle = np.full(K,np.inf)

def Gamma(t):
    
    radius = λmax-λmin/100

    z = radius*np.exp(1j*t)+λmax
    dz = radius*1j*np.exp(1j*t)
    
    return z,dz

for k in range(0,K):
    a_priori_bound_circle[k] = 2*get_a_priori_bound(f,Gamma,[0,np.pi],k,w,λmin,λmax)[0]
    a_posteriori_bound_circle[k] = 2*get_a_posteriori_bound(f,Gamma,[0,np.pi],a_[:k],b_[:k-1],w,λmin,λmax)[0]
    exact_bound_circle[k] = 2*get_exact_bound(f,Gamma,[0,np.pi],a_[:k],b_[:k-1],w,Λ)[0]

In [14]:
a_priori_bound_circle_approx_spectrum = np.full(K,np.inf)
a_posteriori_bound_circle_approx_spectrum = np.full(K,np.inf)
    
def Gamma(t):

    radius = λmax_ - λmin_/100

    z = radius*np.exp(1j*t)+λmax_
    dz = radius*1j*np.exp(1j*t)
    
    return z,dz

for k in range(0,K):
    a_priori_bound_circle_approx_spectrum[k] = 2*get_a_priori_bound(f,Gamma,[0,np.pi],k,w,λmin_,λmax_)[0]
    a_posteriori_bound_circle_approx_spectrum[k] = 2*get_a_posteriori_bound(f,Gamma,[0,np.pi],a_[:k],b_[:k-1],w,λmin_,λmax_)[0]

## $f(x) = \sqrt{x}$, $\Gamma=$ line away from origin

In [15]:
a_priori_bound_wedge = np.full(K,np.inf)
a_posteriori_bound_wedge = np.full(K,np.inf)

def Gamma(t):
    z = 1e-15j - 1*t
    dz = -1
    
    return z,dz

for k in range(1,K):
    a_priori_bound_wedge[k] = 2*get_a_priori_bound(f,Gamma,[0,np.inf],k,w,λmin,λmax)[0]
    a_posteriori_bound_wedge[k] = 2*get_a_posteriori_bound(f,Gamma,[0,np.inf],a_[:k],b_[:k-1],w,λmin,λmax)[0]

In [16]:
a_priori_bound_wedge_approx_spectrum = np.full(K,np.inf)
a_posteriori_bound_wedge_approx_spectrum = np.full(K,np.inf)

def Gamma(t):
    z = 1e-16j - 1*t
    dz = -1
    
    return z,dz

for k in range(1,K):
    a_priori_bound_wedge_approx_spectrum[k] = 2*get_a_priori_bound(f,Gamma,[0,np.inf],k,w,λmin_,λmax_)[0]
    a_posteriori_bound_wedge_approx_spectrum[k] = 2*get_a_posteriori_bound(f,Gamma,[0,np.inf],a_[:k],b_[:k-1],w,λmin_,λmax_)[0]


In [17]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

axs = []
axs.append(fig.add_axes([left, bottom, width2, height]))
axs.append(fig.add_axes([left+hoffset2, bottom, width2, height]))

ks = np.arange(K)

axs[0].plot(ks,(a_priori_bound_circle_approx_spectrum*err_CG),\
        **line_styles['l4'])
#ax.plot(ks,(a_posteriori_bound_circle_approx_spectrum*err_CG),\
#        color=colors[2],marker=markers[2],ms=m_sizes[2],lw=.5)

axs[0].plot(ks,(a_priori_bound_circle*err_CG),\
        **line_styles['l3'])
axs[0].plot(ks,(a_posteriori_bound_circle*err_CG),\
        **line_styles['l2'])

axs[0].plot(ks,err_lanczos,\
        **line_styles['l1'])

axs[0].set_xlabel('number of matvecs: $k$')
axs[0].set_yscale('log')
axs[0].grid(True,linestyle=':',linewidth=.5)


axs[1].plot(ks,(a_priori_bound_wedge_approx_spectrum*err_CG),\
        **line_styles['l4'])
#ax.plot(ks,(a_posteriori_bound_wedge_approx_spectrum*err_CG),\
#        color=colors[2],marker=markers[2],ms=m_sizes[2],lw=.5)

axs[1].plot(ks,(a_priori_bound_wedge*err_CG),\
        **line_styles['l3'])
axs[1].plot(ks,(a_posteriori_bound_wedge*err_CG),\
        **line_styles['l2'])

axs[1].plot(ks,err_lanczos,\
        **line_styles['l1'])

axs[1].set_xlabel('number of matvecs: $k$')
axs[1].set_yscale('log')
axs[1].grid(True,linestyle=':',linewidth=.5)

axs[1].set_yticklabels('')

axs[0].set_ylim(1e-14,5e1)
axs[1].set_ylim(1e-14,5e1)

plt.savefig('imgs/ch7_sqrt_Anorm.pdf')
#plt.close()